In [ ]:
import pandas as pd
import os

In [ ]:
pwd = os.getcwd()

In [ ]:
dataset = pd.read_excel(pwd + '/Data Output.xlsx', sheet_name=('Edited_Data'))
dataset

In [ ]:
dataset_modified = dataset.copy() # Create a copy of the original to work with. As a  backup of the origonal dataser
dataset_modified

In [ ]:
dataset_modified.columns

In [ ]:
columns_to_drop = ['Start Date', 'End Date', 'Email Address',
       'First Name', 'Last Name', 'Custom Data 1']
columns_to_drop

In [ ]:
dataset_modified = dataset_modified.drop(columns=columns_to_drop)
dataset_modified

In [ ]:
id_vars = list(dataset_modified.columns)[:8]
value_vars = list(dataset_modified.columns)[8: ]

In [ ]:
dataset_melted = dataset_modified.melt(id_vars=id_vars,value_vars=value_vars, var_name="Question + Subquestion", value_name="Answer")
dataset_melted

In [ ]:
question_import = pd.read_excel(pwd + '/Data Output.xlsx', sheet_name='Question')
question_import

In [ ]:
questions = question_import.copy()
questions.drop(columns=["Raw Question",	"Raw Subquestion",	"Subquestion"], inplace=True)

In [ ]:
questions

In [ ]:
questions.dropna(inplace=True)

In [ ]:
questions

In [ ]:
dataset_merged = pd.merge(left=dataset_melted, right=questions, how="left",left_on="Question + Subquestion", right_on="Question + Subquestion")
print("Original Data", len(dataset_melted))
print("Merged Data", len(dataset_merged))
dataset_merged

In [ ]:
respondents = dataset_merged[dataset_merged["Answer"].notna()]
respondents = respondents.groupby("Question")["Respondent ID"].nunique().reset_index()
respondents.rename(columns={"Respondent ID":"Respondents"}, inplace=True)
respondents

In [ ]:
dataset_merged_two = pd.merge(left=dataset_merged, right=respondents, how="left",left_on="Question", right_on="Question")
print("Original Data", len(dataset_merged))
print("Merged Data", len(dataset_merged_two))
dataset_merged_two

In [ ]:
same_answer = dataset_merged
same_answer = same_answer.groupby(["Question + Subquestion", "Answer"])["Respondent ID"].nunique().reset_index()
same_answer.rename(columns={'Respondent ID':'Same Answer'},inplace=True)
same_answer

In [ ]:
dataset_merged_three = pd.merge(left=dataset_merged_two, right=same_answer, how="left",left_on=["Question + Subquestion", "Answer"], right_on=["Question + Subquestion", "Answer"])
dataset_merged_three["Same Answer"].fillna(0, inplace=True)
print("Original Data", len(dataset_merged_two))
print("Merged Data", len(dataset_merged_three))
dataset_merged_three

In [ ]:
output = dataset_merged_three.copy()
output.rename(columns={"Identify which division you work in. - Response":"Division Primary", "Identify which division you work in. - Other (please specify)":"Division Secondary", "Which of the following best describes your position level? - Response":"Position", "Which generation are you apart of? - Response":"Generation", "Please select the gender in which you identify. - Response":"Gender", "Which duration range best aligns with your tenure at your company? - Response":"Tenure", "Which of the following best describes your employment type? - Response":"Employment Type"}, inplace=True)
output

In [ ]:
output.to_excel(pwd + "/Final_Output.xlsx", index=False)